In [24]:
import paho.mqtt.client as mqtt
import openpyxl
import json
import time

# Define some constants for the MQTT connection
MQTT_HOST = "broker.mqttdashboard.com"
MQTT_PORT = 1883
MQTT_KEEPALIVE_INTERVAL = 300
MQTT_TOPIC = "CPE314g4_topic2"

# Function to handle message publishing
def on_publish(self, client, userdata):
    print("Message Published...", MQTT_MSG)

# Define a function to handle the on_connect event
def on_connect(client, userdata, flags, rc):
    print("MQTT Connected.")
    
    # publishing a message with an empty payload for ensuring that any new subscribers do not receive any retained messages
    client.publish(MQTT_TOPIC, "", qos=0, retain=True)

# Create a new MQTT client
client = mqtt.Client()

# Set the on_publish callback for the client
client.on_publish = on_publish

# Connect to the MQTT broker
client.connect(MQTT_HOST,MQTT_PORT,MQTT_KEEPALIVE_INTERVAL)

# Set the on_connect and on_message callback functions for the client
client.on_connect = on_connect

# Define node_id
node_id = "0002"

# Open the Excel file
workbook = openpyxl.load_workbook('SampleInput.xlsx')

# Select the worksheet you want to read data from
worksheet = workbook['Sheet1']

# Declare variables
count = 0
chunk_size = 200

# Store data from the worksheet
for row in worksheet.iter_rows(min_row=2, values_only=True):
    datetime = str(row[0])
    humidity = str(row[1])
    temperature = str(row[2])
    thermal_array = str(row[3])
    
    # Create data object for sending to MQTT
    data = {
        "r": str(count),
        "n": node_id,
        "c": 0,
        "p": datetime + "/" +  humidity + "/" + temperature,
        "l": len(datetime) + len(humidity) + len(temperature) + len(thermal_array)
    }

    # Convert data object to JSON and publish to MQTT broker
    MQTT_MSG = json.dumps(data)
    client.publish(MQTT_TOPIC, MQTT_MSG, qos=0, retain=True)
    
    # Break up the thermal array into chunks and send each chunk to MQTT broker
    for j in range(0, len(thermal_array), chunk_size):
        chunk = thermal_array[j:j+chunk_size]
        data = {
            "r": str(count),
            "n": node_id,
            "c": int(j/chunk_size+1),
            "p": chunk
        }
        MQTT_MSG = json.dumps(data)
        client.publish(MQTT_TOPIC, MQTT_MSG, qos=0, retain=True)
    count += 1
    # sleep for 3 min
    time.sleep(1) 

Message Published... {"r": "0", "n": "0001", "c": 0, "p": "2022-11-08 12:08:00/55.82/29.16", "l": 3868}
Message Published... {"r": "0", "n": "0001", "c": 1, "p": "12.9,12.0,13.0,13.2,13.5,12.9,13.2,12.9,13.7,13.0,13.0,13.1,13.5,12.9,13.6,13.5,13.4,13.3,13.6,13.7,13.8,13.2,13.6,13.6,13.3,13.1,13.7,13.2,13.4,13.2,13.1,13.3,13.7,13.2,13.2,13.1,13.5,13.2,12.9,13.5,"}
Message Published... {"r": "0", "n": "0001", "c": 2, "p": "13.4,13.3,13.4,13.8,13.1,13.1,13.2,13.6,13.3,13.2,13.4,13.7,13.5,13.2,13.2,13.7,13.6,13.6,12.9,13.2,13.6,13.2,13.2,13.2,13.2,12.6,13.8,13.1,13.1,13.4,13.4,13.3,13.7,13.0,13.7,13.1,13.6,13.3,13.7,13.2,"}
Message Published... {"r": "0", "n": "0001", "c": 3, "p": "13.6,13.1,13.8,13.2,13.6,13.2,13.7,13.4,13.6,13.1,13.5,13.9,13.5,13.2,13.2,13.1,12.6,13.2,13.1,13.4,13.5,13.3,13.2,13.5,13.6,13.5,13.1,13.4,13.5,13.4,13.3,13.8,13.3,13.4,13.4,13.7,13.7,13.2,13.6,13.5,"}
Message Published... {"r": "0", "n": "0001", "c": 4, "p": "13.5,13.2,13.3,13.4,13.2,13.5,13.2,13.5,13.3,12.7,1